# Net Zero Coalition - Embodied Carbon Work Package

## Graphic summary of pipeline projects
The code in this notebook will provide a graphic summary of projects selected for further analysis from the Infrastructure and Projects Authority National Infrastructure and Construction Pipeline.

Code was authored by Dr Jannik Giesekam at University of Leeds in March 2020 - contact J.Giesekam@leeds.ac.uk for more details

In [144]:
# Import necessary packages
import yaml
import pandas as pd
import re
from bokeh.layouts import layout, column
from bokeh.models import ColumnDataSource, Select, HoverTool, BoxAnnotation, Span, Label, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap

## Import data

Import selected projects that were previously filtered

In [145]:
projects = pd.read_csv('selected_projects.csv',engine='python',encoding='utf-8-sig') # load data

## Prepare and visualise data

Put into groups and create interactive graphic for EDA

In [146]:
# Tidy data

# Start by renaming columns to formats that bokeh can handle
projects.rename(columns={'Total Capex Cost all funding (£m)':'Capex', 
                         'Project / Programme Name':'Project',
                         'Project Summary':'Description',
                         'Key Client':'Client',
                         'Scheme Status':'Status',
                         'Start of Works / Construction (Projected)':'Start_Date',
                         'Date in Service (Projected)':'Date_In_Service'
                        },inplace=True)

# Set data types
projects['Sector'] = projects['Sector'].astype(str)
projects['Sub-Sector'] = projects['Sub-Sector'].astype(str)
projects['Project'] = projects['Project'].astype(str)
projects['Description'] = projects['Description'].astype(str)

# Clean numeric values
projects['Capex'] = projects['Capex'].str.replace(',','') # remove commas from capex values
projects['Capex'] = pd.to_numeric(projects['Capex'], errors='coerce')
projects['Capex'] = projects['Capex']*1000000 # convert to £ from millions

# Dates
# Clean and convert dates to datetime
projects['Start_Date_Clean'] = projects['Start_Date'] # create cleaning variant of date whilst preserving original
# replace ranges with initial start date
two_year_range_pat = re.compile(r'/\d\d$') # expression for common data entry format e.g. 2018/19
alt_pat = re.compile(r'-\d*') # expression for common data entry format e.g. 2018-19
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace(two_year_range_pat,'')
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace(alt_pat,'')
# replace text values
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace('Active Programme','2020') # Insert 2020 for active
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace('Started','2020') # Insert 2020 for started
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace('Ongoing','2020') # Insert 2020 for ongoing
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace('Various','2020') # Insert 2020 for various
projects['Start_Date_Clean'] = projects['Start_Date_Clean'].str.replace('TBC','2020') # Insert 2021 for TBC
# convert two remaining common formats
projects['Start_Date_Clean'].fillna('2020', inplace=True) # fill blanks with 2020
projects['Start_Date_Cleaned'] = pd.to_datetime(projects['Start_Date_Clean'], format='%Y',errors='coerce') # Convert dates to datetime that follow 2019 format
mask = projects['Start_Date_Cleaned'].isnull() # create mask for other format
projects.loc[mask, 'Start_Date_Cleaned'] = pd.to_datetime(projects[mask]['Start_Date_Clean'], format = '%b %y', errors='coerce') # Convert dates to datetime that follow Oct 2016 format
# Clean and convert date in service dates to datetime
projects['Date_In_Service_Clean'] = projects['Date_In_Service'] # create cleaning variant of date whilst preserving original
# replace ranges with initial start date
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace(two_year_range_pat,'')
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace(alt_pat,'')
# replace text values
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('Ongoing','2020') # Insert 2020 for ongoing
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('Various','2021') # Insert 2020 for various
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('TBC','2021') # Insert 2021 for TBC
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('2021/22 ','2021') # correction
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('21','2021') # correction
projects['Date_In_Service_Clean'] = projects['Date_In_Service_Clean'].str.replace('202021','2021') # correction
# convert two remaining common formats
projects['Date_In_Service_Cleaned'] = pd.to_datetime(projects['Date_In_Service_Clean'],errors='coerce') # Convert dates to datetime that follow 2019 format
projects['Date_In_Service_Cleaned'].fillna(pd.Timestamp('2025-01-01'), inplace=True) # fill blanks with 2025

# Other clean up bits and bobs
projects['Sub-Sector'] = projects['Sub-Sector'].str.strip() # remove spaces
projects.loc[projects['Sub-Sector']=='Electricity transmission','Sub-Sector'] = 'Electricity Transmission' # remove difference in case
projects.loc[projects['Status']=='Active programme','Status'] = 'Active Programme' # remove difference in case

# Create short description column
projects['Short_Description'] = projects['Description'].str[0:150] # use first 150 characters

In [151]:
# Plot bar charts showing aggregated headline data by sector, sub-sector and scheme status
output_notebook()

# set colormap based on sector
index_cmap = factor_cmap('Sector_Sub-Sector', palette=Category10[7], factors=sorted(projects['Sector'].unique()), end=1)

# Plot number of projects by sector and sub-sector
group = projects.groupby(by=['Sector', 'Sub-Sector'])
p = figure(plot_width=900, plot_height=500, title="Count of projects by category", x_range=group)
p.vbar(x='Sector_Sub-Sector', top='Capex_count', source=group, width=1, 
       fill_color=index_cmap,line_color="black")
p.y_range.start = 0
p.x_range.range_padding = 0.05
p.xgrid.grid_line_color = None
p.yaxis.axis_label = "Number of projects"
p.xaxis.major_label_orientation = 1.2

# Plot total capex of projects by sector and sub-sector
group2 = projects.groupby(by=['Sector','Sub-Sector']).sum()
p2 = figure(plot_width=900, plot_height=500, title="Capex of projects by category", x_range=group)
p2.vbar(x='Sector_Sub-Sector', top='Capex', source=group2, width=1, 
       fill_color=index_cmap,line_color="black")
p2.y_range.start = 0
p2.x_range.range_padding = 0.05
p2.xgrid.grid_line_color = None
p2.yaxis.axis_label = "Total Capex £ (where stated)"
p2.xaxis.major_label_orientation = 1.2
p2.yaxis[0].formatter = NumeralTickFormatter(format="(0 a)")

# Plot by sector and project status
index_cmap2 = factor_cmap('Sector_Status', palette=Category10[7], factors=sorted(projects['Sector'].unique()), end=1)
group3 = projects.groupby(by=['Sector','Status']).sum()
group3b = projects.groupby(by=['Sector','Status'])
p3 = figure(plot_width=900, plot_height=500, title="Capex of projects by scheme status", x_range=group3b)
p3.vbar(x='Sector_Status', top='Capex', source=group3, width=1,
        fill_color=index_cmap2, line_color="black")
p3.y_range.start = 0
p3.x_range.range_padding = 0.05
p3.xgrid.grid_line_color = None
p3.yaxis.axis_label = "Total Capex £ (where stated)"
p3.xaxis.major_label_orientation = 1.2
p3.yaxis[0].formatter = NumeralTickFormatter(format="(0 a)")

def modify_doc(doc):        
    # Arrange layout of plot
    sizing_mode = 'fixed'
    l = layout([p, p2, p3], sizing_mode=sizing_mode)
        
    doc.add_root(l)    
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                toolbar_location: above
            Grid:
                grid_line_color: white
    """))
show(modify_doc)

Loading BokehJS ...

C:\Users\Jannik Giesekam\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


In [159]:
# Create EDA graphic with circle for each project and dropdowns to select by sector with tooltip showing key info on hover
          
# Extract style list
sector_list = projects.Sector.unique().tolist()
sector_list.append('All') # append an All sectors option
            
# Generate custom tooltips
TOOLTIPS_EDA = """
    <div style="margin:5px" >
        <div>
            <span style="font-size: 15px; font-weight: bold;">@Project</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold;">£@Capex{0 a} - @Client</span>
        </div>       
        <div>
            <span style="color: #31476b">Status:</span>&nbsp<span>@Status</span>
        </div>
        <div>
            <span style="color: #31476b">Start of Works:</span>&nbsp<span>@Start_Date</span>
        </div>
        <div>
            <span style="color: #31476b">Date in Service (Projected):</span>&nbsp<span>@Date_In_Service</span>
        </div>        
        <div>        
            <span style="font-size: 10px">@Short_Description</span>
        </div>
    </div>
"""
hover = HoverTool(tooltips=TOOLTIPS_EDA)  

# set colormap based on sector
status_cmap = factor_cmap('Status', palette=Category10[8], factors=sorted(projects['Status'].unique()), end=1)

output_notebook()

# Map selection choices to df columns
axis_map = {
    "Start date": "Start_Date_Cleaned",
    "Date in service":'Date_In_Service_Cleaned'
}

# Create controls
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="Start date")
subset_selection = Select(title="Sector", value="All", options=sector_list)

# Set plot dimensions, data source and glyphs
p = figure(plot_height=600, plot_width=600, x_axis_type='datetime', title="Projects", tools=[hover,'pan','wheel_zoom','box_zoom','save','reset'])
source = ColumnDataSource(data=dict(x=[], y=[], color=[]))
p.circle(x="x", y="y", source=source, color=status_cmap, size=15, line_color=None)

# Set initial plot data
df = projects
source.data = dict(
    x=df['Start_Date_Cleaned'],
    y=df['Capex'],
    Project=df['Project'],
    Status=df['Status'],
    Short_Description=df['Short_Description'],
    Client=df['Client'],
    Capex=df['Capex'],
    Start_Date_Cleaned=df['Start_Date_Cleaned'],
    Start_Date=df['Start_Date'],
    Date_In_Service_Cleaned=df['Date_In_Service_Cleaned'],
    Date_In_Service=df['Date_In_Service']
)
p.yaxis.axis_label = 'Capex £'
p.yaxis.formatter=NumeralTickFormatter(format="0 a")

# Create function to select projects based on sector selection
def select_subset():
    subset_selection_val = subset_selection.value
    selected = projects
    if (subset_selection_val != "All"):
        selected = selected[selected.Sector.str.contains(subset_selection_val)==True]
    return selected


def modify_doc(doc):
    
    def update():
        df = select_subset()
        
        # Update axis labels
        x_name = axis_map[x_axis.value]
        p.xaxis.axis_label = x_axis.value
        
        # Update source data to new values
        source.data = dict(
            x=df[x_name],
            y=df['Capex'],
            Project=df['Project'],
            Status=df['Status'],
            Short_Description=df['Short_Description'],
            Client=df['Client'],
            Capex=df['Capex'],
            Start_Date=df['Start_Date'],
            Start_Date_Cleaned=df['Start_Date_Cleaned'],
            Date_In_Service=df['Date_In_Service'],
            Date_In_Service_Cleaned=df['Date_In_Service_Cleaned']
            )
    
    # Update plot when values for controls are changed
    controls = [subset_selection, x_axis]
    for control in controls:
        control.on_change('value', lambda attr, old, new: update())    
    
    # Arrange layout of plot
    sizing_mode = 'fixed'
    inputs = column(*controls, sizing_mode=sizing_mode)
    l = layout([
        [inputs, p],
    ], sizing_mode=sizing_mode)
        
    doc.add_root(l)    
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#DDDDDD"
                outline_line_color: white
                toolbar_location: above
            Grid:
                grid_line_color: white
    """))
show(modify_doc)

Loading BokehJS ...

C:\Users\Jannik Giesekam\Anaconda3\lib\site-packages\ipykernel_launcher.py:124: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
